# 🧠 Phi-3 RAG — Bengali Q&A + Quiz Generation
### NCTB Class 3 AI Tutor | Group 11 — BUET CSE

**এই notebook এ যা হবে:**
1. আগের notebook এর FAISS index load করা
2. Phi-3 Mini model download ও load করা (Kaggle T4 GPU তে চলবে)
3. Bengali/English প্রশ্ন → NCTB context retrieve → Phi-3 উত্তর
4. AI দিয়ে Quiz generate করা
5. Interactive chat loop test করা

---
**⚠️ Setup:**
- আগের OCR notebook এর output dataset হিসেবে add করো
- GPU T4 x2 ON রাখো
- Internet ON রাখো (model download এর জন্য)

## Step 1: Install Libraries

In [ ]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q sentence-transformers faiss-cpu
!pip install -q langchain

print('✅ সব install হয়েছে!')

## Step 2: FAISS Index Load করা (আগের notebook থেকে)

In [ ]:
import os, pickle, json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch

# ── Paths ──────────────────────────────────────────────────────
# আগের notebook এর output Kaggle Dataset হিসেবে add করতে হবে
# Dataset name যদি 'nctb-rag-output' হয়:
BASE = '/kaggle/input/nctb-rag-output'  # ← তোমার dataset name অনুযায়ী change করো

# Fallback: working directory তে থাকলে সেখান থেকে নাও
if not os.path.exists(BASE):
    BASE = '/kaggle/working'
    print('⚠️  Input dataset পাওয়া যায়নি — working directory থেকে load হচ্ছে')

FAISS_PATH  = f'{BASE}/faiss_index/bangla_class3.faiss'
CHUNKS_PATH = f'{BASE}/faiss_index/chunks.pkl'

# Load FAISS index
print('🔄 FAISS index load হচ্ছে...')
INDEX = faiss.read_index(FAISS_PATH)

# Load text chunks
with open(CHUNKS_PATH, 'rb') as f:
    CHUNKS = pickle.load(f)

print(f'✅ Index loaded!')
print(f'   Vectors: {INDEX.ntotal}')
print(f'   Chunks: {len(CHUNKS)}')
print(f'   Sample chunk: {CHUNKS[5][:150]}')

## Step 3: Embedding Model Load (Retrieval এর জন্য)

In [ ]:
print('🔄 Embedding model load হচ্ছে...')

EMBED_MODEL = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

def retrieve(question, k=3):
    """Student এর question এর জন্য relevant NCTB chunks খুঁজে আনা।"""
    q_emb = EMBED_MODEL.encode(
        [question],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')
    
    scores, indices = INDEX.search(q_emb, k)
    
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx >= 0:
            results.append({
                'text': CHUNKS[idx],
                'score': float(score)
            })
    return results

# Quick test
test = retrieve('বন্ধু কাকে বলে?')
print(f'\n✅ Retrieval test:')
for i, r in enumerate(test, 1):
    print(f'  [{i}] score={r["score"]:.3f}: {r["text"][:120]}...')

## Step 4: Phi-3 Mini Load করা

**Phi-3 Mini (3.8B)** — Microsoft এর lightweight model।  
Kaggle T4 GPU তে 4-bit quantization দিয়ে চলবে (~4GB VRAM)।

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

MODEL_ID = 'microsoft/Phi-3-mini-4k-instruct'

print(f'🔄 Phi-3 Mini load হচ্ছে...')
print(f'   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')
print(f'   VRAM: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB')
print('   (প্রথমবার ~৫ মিনিট লাগবে — model download হচ্ছে)\n')

# 4-bit quantization config — VRAM কম লাগবে, speed বেশি
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

# Tokenizer
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# Model
PHI3 = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map='auto',           # GPU/CPU automatically assign করবে
    trust_remote_code=True,
    torch_dtype=torch.float16,
    attn_implementation='eager'  # Flash attention issue avoid করতে
)

PHI3.eval()
print('\n✅ Phi-3 Mini ready!')
print(f'   Memory used: {torch.cuda.memory_allocated() // 1024**2} MB')

## Step 5: RAG Generation Function

In [ ]:
def generate_answer(question, mode='chat', max_new_tokens=300):
    """
    Student এর question এর জন্য NCTB-grounded উত্তর generate করা।
    
    mode:
      'chat' → সহজ Bengali উত্তর
      'quiz' → MCQ questions JSON format এ
      'explain' → বিস্তারিত ব্যাখ্যা
    """
    # Step 1: Relevant context retrieve করো
    chunks = retrieve(question, k=3)
    context = '\n\n'.join(
        f'[অংশ {i+1}]: {c["text"]}'
        for i, c in enumerate(chunks)
    )
    
    # Step 2: Mode অনুযায়ী prompt তৈরি
    if mode == 'chat':
        system = (
            'তুমি একটি AI শিক্ষক। বাংলাদেশের তৃতীয় শ্রেণির ছাত্রছাত্রীদের '
            'NCTB পাঠ্যবই অনুযায়ী সাহায্য করো। '
            'সহজ, বন্ধুত্বপূর্ণ বাংলায় ছোট উত্তর দাও। '
            'শুধু নিচের বইয়ের অংশ থেকে উত্তর দাও।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context}\n\nপ্রশ্ন: {question}'

    elif mode == 'quiz':
        system = (
            'তুমি একজন শিক্ষক। NCTB পাঠ্যবইয়ের তথ্য থেকে '
            'তৃতীয় শ্রেণির ছাত্রদের জন্য MCQ তৈরি করো। '
            'শুধু JSON format এ output দাও, অন্য কিছু লিখবে না।'
        )
        user = (
            f'পাঠ্যবইয়ের অংশ:\n{context}\n\n'
            f'"{question}" বিষয়ে ৩টি সহজ MCQ তৈরি করো।\n'
            'Format:\n'
            '{"questions": [{"question": "...", '
            '"options": ["ক) ...", "খ) ...", "গ) ...", "ঘ) ..."], '
            '"correct": "ক", "explanation": "..."}]}'
        )

    elif mode == 'explain':
        system = (
            'তুমি একটি AI শিক্ষক। NCTB পাঠ্যবই অনুযায়ী '
            'তৃতীয় শ্রেণির ছাত্রদের জন্য বিষয়টি সহজভাবে ব্যাখ্যা করো। '
            'উদাহরণ দিয়ে বোঝাও।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context}\n\nব্যাখ্যা করো: {question}'

    # Step 3: Phi-3 format এ prompt তৈরি
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user',   'content': user}
    ]
    
    prompt = TOKENIZER.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Step 4: Tokenize
    inputs = TOKENIZER(
        prompt,
        return_tensors='pt',
        truncation=True,
        max_length=2048
    ).to(PHI3.device)
    
    # Step 5: Generate
    with torch.no_grad():
        output = PHI3.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.3,     # Low temp = more focused/accurate
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=TOKENIZER.eos_token_id
        )
    
    # Step 6: Decode — শুধু নতুন generated text
    generated = output[0][inputs['input_ids'].shape[1]:]
    answer = TOKENIZER.decode(generated, skip_special_tokens=True).strip()
    
    return answer, context  # context ও return করি debug এর জন্য


print('✅ RAG generation function ready!')

## Step 6: Bengali Q&A Test

In [ ]:
# Test questions — Class 3 বাংলা বই থেকে
test_questions = [
    'তোমার বন্ধু কে?',
    'পরিবারে কারা থাকে?',
    'বাংলাদেশের কথা বলো।',
    'What is this lesson about?',  # English test
]

print('=' * 60)
print('🧪 Bengali Q&A Test — Phi-3 + NCTB RAG')
print('=' * 60)

for q in test_questions:
    print(f'\n❓ প্রশ্ন: {q}')
    print('🤔 উত্তর generate হচ্ছে...')
    
    answer, ctx = generate_answer(q, mode='chat')
    
    print(f'🤖 AI উত্তর:\n{answer}')
    print(f'\n📚 Retrieved context (প্রথম ১০০ chars): {ctx[:100]}...')
    print('-' * 60)

## Step 7: Quiz Generation Test

In [ ]:
import json as json_lib
import re

def generate_quiz(topic, num_q=3):
    """Topic এর উপর MCQ quiz generate করা।"""
    answer, _ = generate_answer(topic, mode='quiz', max_new_tokens=500)
    
    # JSON parse করার চেষ্টা
    try:
        # JSON block extract করো
        json_match = re.search(r'\{.*\}', answer, re.DOTALL)
        if json_match:
            quiz_data = json_lib.loads(json_match.group())
            return quiz_data
    except:
        pass
    
    # JSON parse না হলে raw text return করো
    return {'raw': answer}


def display_quiz(quiz_data):
    """Quiz সুন্দরভাবে display করা।"""
    if 'raw' in quiz_data:
        print(quiz_data['raw'])
        return
    
    questions = quiz_data.get('questions', [])
    for i, q in enumerate(questions, 1):
        print(f'\n{i}. {q.get("question", "")}')
        for opt in q.get('options', []):
            print(f'   {opt}')
        print(f'   ✅ সঠিক উত্তর: {q.get("correct", "")}')
        if q.get('explanation'):
            print(f'   💡 ব্যাখ্যা: {q["explanation"]}')


# Test
print('=' * 60)
print('📝 Quiz Generation Test')
print('=' * 60)

topics = ['বন্ধুত্ব', 'পরিবার']

for topic in topics:
    print(f'\n🎯 Topic: {topic}')
    print('Quiz generate হচ্ছে...')
    quiz = generate_quiz(topic)
    display_quiz(quiz)
    print('-' * 60)

## Step 8: Interactive Chat Loop (Full Pipeline Test)

In [ ]:
def detect_language(text):
    """Bengali নাকি English সেটা detect করা।"""
    bn_chars = sum(1 for c in text if '\u0980' <= c <= '\u09FF')
    return 'bengali' if bn_chars > len(text) * 0.2 else 'english'

def ai_tutor_response(student_input, conversation_history=[]):
    """
    Full AI tutor pipeline:
    Student input → Language detect → RAG → Phi-3 → Response
    """
    lang = detect_language(student_input)
    
    # Quiz request detect করা
    quiz_keywords = ['quiz', 'প্রশ্ন করো', 'পরীক্ষা', 'mcq', 'কুইজ']
    is_quiz = any(kw in student_input.lower() for kw in quiz_keywords)
    
    if is_quiz:
        # Topic extract করো এবং quiz generate করো
        topic = student_input.replace('quiz', '').replace('কুইজ', '').replace('প্রশ্ন করো', '').strip()
        if not topic:
            topic = 'বাংলা পাঠ'
        answer, ctx = generate_answer(topic, mode='quiz', max_new_tokens=500)
    else:
        answer, ctx = generate_answer(student_input, mode='chat')
    
    return {
        'answer': answer,
        'language': lang,
        'mode': 'quiz' if is_quiz else 'chat',
        'context_used': ctx[:200] + '...'
    }


# Simulate a conversation
print('=' * 60)
print('💬 AI Tutor — Simulated Conversation')
print('=' * 60)

conversations = [
    'আমাকে বন্ধুত্ব সম্পর্কে বলো।',
    'বন্ধুত্ব নিয়ে কুইজ দাও।',
    'Tell me about family.',
]

for msg in conversations:
    print(f'\n👦 Student: {msg}')
    response = ai_tutor_response(msg)
    print(f'🤖 AI Tutor ({response["mode"]} | {response["language"]}):')
    print(response['answer'])
    print('-' * 60)

## Step 9: Performance Summary

In [ ]:
import time

# Speed test
test_q = 'পরিবার কী?'
start = time.time()
ans, _ = generate_answer(test_q, mode='chat')
elapsed = time.time() - start

print('=' * 60)
print('📊 Performance Summary')
print('=' * 60)
print(f'''
🧠 Model       : Phi-3 Mini 4k (4-bit quantized)
📚 Knowledge   : NCTB Class 3 Bengali (107 pages, 190 chunks)
🔍 Retrieval   : FAISS + Multilingual MiniLM
⚡ Speed        : {elapsed:.1f}s per response
🌐 Languages   : Bengali + English
📝 Modes       : Chat, Quiz, Explain

✅ Ready for:
   → Whisper STT integration (voice input)
   → Edge-TTS integration (voice output)
   → Django REST API wrapping
   → Flutter app connection
''')
print('=' * 60)